In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
def load_and_normalize_dataset():
    DataSet = load_breast_cancer()
    DataFrame = pd.DataFrame(DataSet.data, columns= DataSet.feature_names)
    DataFrame['Target'] = DataSet.target

    normalizer = StandardScaler()
    normalized_Input_Fields = normalizer.fit_transform (DataFrame.iloc[:,: len(DataSet.feature_names)])
    return normalized_Input_Fields, DataFrame['Target'].values


def Split_into_K_Folds(K, X,Y):
    print (len(X))

    indices = np.arange(len(X))
    np.random.shuffle(indices)
    
    X = X[indices]
    Y = Y[indices]
    
    size_of_folds = [len(X)//K for i in range(K)]

    for i in range(len(X)%K ):
        size_of_folds[i] += 1 
    
    print (size_of_folds)
    S_index = 0
    E_index = 0

    Folds = []

    for i in range (K):
        S_index =  E_index
        E_index = S_index + size_of_folds[i]
        Folds.append((X[S_index:E_index],Y[S_index:E_index]))
    
    return Folds

X,Y = load_and_normalize_dataset()
Folds = Split_into_K_Folds (5, X,Y)
print(Folds[0])


569
[[-0.48205495 -0.53281658 -0.55067316 ... -1.31384558 -1.61738541
  -1.36465979]
 [-0.40537112 -1.65679    -0.45675951 ... -0.49282745 -0.20344515
   0.59262688]
 [-0.81435156  0.15599529 -0.75168134 ... -0.32685754  0.08775536
  -0.76783144]
 ...
 [-0.51329651 -1.60559452 -0.54037561 ... -0.31513306 -0.11932056
  -0.89972108]
 [-0.33152743 -0.23262492 -0.32083185 ... -0.73888926  0.49543607
  -0.63594179]
 [-0.54453807 -1.20999311 -0.54325893 ... -0.35335182 -0.36198765
   0.23796565]]
[1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0
 0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1
 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 0 1
 0 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 0 1 0 0 0 1 1 0 1 1 1 0 0
 0 0 0 1 1 1 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1
 1 0 1 1 1 0 0 0 1 1 1 0 1 0 1 1 1 1 0 0 1 0 1 1 1 0 0 0 1 0 1 1 1 0 0 1 0
 0 1 1 1 1 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0 0 0 0 1 0 0 1